# Modular NN4MG Mesh Generation Demo

This notebook mirrors the original `MeshGenDemo.ipynb`, but uses the modularized
Python package in `NN4MG/nn4mg` and the training script logic.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt

from nn4mg import (
    build_boundary_data,
    extract_boundaries,
    init_model_weights,
    load_grid_csv,
    M_fun_torch,
    Model,
    plot_grid,
    train,
)

## Load grid data

Ensure `x1.csv` and `x2.csv` exist in the working directory (or update paths below).

In [ ]:
X1, X2 = load_grid_csv("x1.csv", "x2.csv")
Nx1, Nx2 = np.shape(X1)

fig, _ = plot_grid(X1, X2, title="Initial grid")
plt.show()

## Build boundary data and model

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dtype = torch.float32

south_np, north_np, west_np, east_np = extract_boundaries(X1, X2)
boundary = build_boundary_data(
    south_np, north_np, west_np, east_np, device=device, dtype=dtype
)

net = Model(width=128, depth=4)
init_model_weights(net)

## Train

Set `plot_every` to 0 to disable training plots.

In [ ]:
net, best = train(
    net,
    X1,
    X2,
    boundary,
    M_fun_torch,
    steps=40,
    N_bdry=256,
    lr=5e-4,
    plot_every=0,
    device=device,
    dtype=dtype,
)

## Visualize final grid

In [ ]:
xy_int = np.concatenate((X1.flatten().reshape(-1, 1), X2.flatten().reshape(-1, 1)), axis=1)
xy_int = torch.from_numpy(xy_int).float().to(device=device, dtype=dtype)

with torch.no_grad():
    out = net(xy_int)

X = torch.reshape(out[:, 0], (Nx2, Nx1)).detach().cpu().numpy()
Y = torch.reshape(out[:, 1], (Nx2, Nx1)).detach().cpu().numpy()

fig, _ = plot_grid(X, Y, title="Deformed grid")
plt.show()